<a href="https://colab.research.google.com/github/CGHAI/colabs/blob/main/llama_whisper_gpt4all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown Clone and build llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!make

Cloning into 'llama.cpp'...
remote: Enumerating objects: 1185, done.
remote: Counting objects: 100% (1185/1185), done.
remote: Compressing objects: 100% (465/465), done.
remote: Total 1185 (delta 727), reused 1120 (delta 693), pack-reused 0
Receiving objects: 100% (1185/1185), 1.04 MiB | 13.46 MiB/s, done.
Resolving deltas: 100% (727/727), done.
/content/llama.cpp
I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -DNDEBUG -std=c11   -fPIC -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -Wno-unused-function -pthread -mavx -mavx2 -mfma -mf16c -msse3
I CXXFLAGS: -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -pthread
I LDFLAGS:  
I CC:       cc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
I CXX:      g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0

cc  -I.              -O3 -DNDEBUG -std=c11   -fPIC -Wall -Wextra -Wpedantic -Wcast

In [2]:
#@markdown Obtain the original LLaMA model weights and place them in ./models<br>
#@markdown `ls ./models`<br>
#@markdown 65B 30B 13B 7B tokenizer_checklist.chk tokenizer.model<br>
!ls ./models

ggml-vocab.bin


In [4]:
#@markdown Clone and build whisper.cpp
%cd ..
!git clone https://github.com/CGHAI/whisper.cpp.git
%cd whisper.cpp
!make

/content
Cloning into 'whisper.cpp'...
remote: Enumerating objects: 2917, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 2917 (delta 40), reused 57 (delta 27), pack-reused 2832
Receiving objects: 100% (2917/2917), 5.05 MiB | 20.02 MiB/s, done.
Resolving deltas: 100% (1795/1795), done.
/content/whisper.cpp
I whisper.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -DNDEBUG -std=c11   -fPIC -pthread -mavx -mavx2 -mfma -mf16c -msse3
I CXXFLAGS: -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -pthread
I LDFLAGS:  
I CC:       cc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
I CXX:      g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0

cc  -I.              -O3 -DNDEBUG -std=c11   -fPIC -pthread -mavx -mavx2 -mfma -mf16c -msse3   -c ggml.c -o ggml.o
g++ -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -pthread -c whisper.cpp -o whisper.o
g++ -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPI

In [5]:
#@markdown Download ggml model
%cd models
!./download-ggml-model.sh base.en

/content/whisper.cpp/models
ggml-base.en.bin    100%[===================>] 141.11M   163MB/s    in 0.9s    
Done! Model 'base.en' saved in 'models/ggml-base.en.bin'
You can now use it like this:

  $ ./main -m models/ggml-base.en.bin -f samples/jfk.wav



In [6]:
#@markdown Execute the example
%cd ..
!./main -m models/ggml-base.en.bin -f samples/jfk.wav

/content/whisper.cpp
whisper_init_from_file_no_state: loading model from 'models/ggml-base.en.bin'
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51864
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 512
whisper_model_load: n_audio_head  = 8
whisper_model_load: n_audio_layer = 6
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 512
whisper_model_load: n_text_head   = 8
whisper_model_load: n_text_layer  = 6
whisper_model_load: n_mels        = 80
whisper_model_load: f16           = 1
whisper_model_load: type          = 2
whisper_model_load: mem required  =  218.00 MB (+    6.00 MB per decoder)
whisper_model_load: adding 1607 extra tokens
whisper_model_load: model ctx     =  140.60 MB
whisper_model_load: model size    =  140.54 MB
whisper_init_state: kv self size  =    5.25 MB
whisper_init_state: kv cross size =   17.58 MB

system_info: n_threads = 2 / 2 | AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | A

In [ ]:
# install Python dependencies
!pip install torch numpy sentencepiece

In [ ]:
# convert the 7B model to ggml FP16 format
python3 convert-pth-to-ggml.py models/7B/ 1

In [ ]:
# quantize the model to 4-bits
python3 quantize.py 7B

In [ ]:
# run the inference
./main -m ./models/7B/ggml-model-q4_0.bin -n 128